In [1]:
pip install requests beautifulsoup4 reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.5 MB/s eta 0:00:00


In [3]:
!apt-get update
!apt-get install -y tesseract-ocr


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [945 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,240 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,425 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages 

In [4]:
!pip install pytesseract



In [6]:
import os
import requests
from PIL import Image
from io import BytesIO
from bs4 import BeautifulSoup
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from urllib.parse import urljoin, urlparse
import pytesseract

# Function to scrape website content
def scrape_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

# Function to extract title, headers, paragraphs, and images from the scraped content
def extract_data(soup):
    title = soup.title.string if soup.title else 'No Title'
    headers = [header.get_text() for header in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]
    paragraphs = [p.get_text() for p in soup.find_all('p')]
    images = [img['src'] for img in soup.find_all('img', src=True)]
    return title, headers, paragraphs, images

# Function to extract text from an image
def extract_text_from_image(image_url):
    try:
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content))
        text = pytesseract.image_to_string(image)
        return text.strip()
    except Exception as e:
        print(f"Error extracting text from image {image_url}: {e}")
        return None

# Function to generate a single PDF with aggregated content using reportlab
def generate_pdf(title, headers, paragraphs, images, output_filename='PC_Hardware_News_Compilation.pdf'):
    styles = getSampleStyleSheet()
    title_style = styles['Title']
    header_style = styles['Heading2']
    normal_style = styles['Normal']

    pdf = SimpleDocTemplate(output_filename, pagesize=letter)
    story = []

    # Title Formatting
    story.append(Paragraph(title, title_style))
    story.append(Spacer(1, 12))

    # Header Formatting
    for header in headers:
        story.append(Paragraph(header, header_style))
        story.append(Spacer(1, 6))

    # Paragraph Formatting
    for paragraph in paragraphs:
        story.append(Paragraph(paragraph, normal_style))
        story.append(Spacer(1, 12))

    # Image Text Extraction
    for image_url in images:
        image_text = extract_text_from_image(image_url)
        if image_text:
            story.append(Paragraph(f"Image Text: {image_text}", normal_style))
            story.append(Spacer(1, 12))

    # Build the PDF
    pdf.build(story)
    print(f"PDF generated successfully and saved as {output_filename}!")

# Main script
if __name__ == '__main__':
    urls = [
        'https://gamersnexus.net/'  # Add more URLs as needed
    ]

    all_headers = []
    all_paragraphs = []
    all_images = []
    visited_urls = set()
    page_limit = 600  # Limit the total number of pages to scrape
    total_pages_scraped = 0

    def scrape_all(url, depth=0, max_depth=20):
        global total_pages_scraped
        if url in visited_urls or depth > max_depth or total_pages_scraped >= page_limit:
            return
        visited_urls.add(url)

        print(f"Scraping {url}")
        soup = scrape_website(url)
        if soup:
            title, headers, paragraphs, images = extract_data(soup)
            all_headers.extend(headers)
            all_paragraphs.extend(paragraphs)
            all_images.extend(images)
            total_pages_scraped += 1

            # Collect links
            links = get_links(soup, url)
            for link in links:
                if total_pages_scraped >= page_limit:
                    break
                scrape_all(link, depth + 1)  # Recursively scrape internal links with increased depth

    # Start scraping from each URL
    for start_url in urls:
        scrape_all(start_url)

    # Generate a single PDF with aggregated content
    generate_pdf('Aggregated Content from Multiple Pages', all_headers, all_paragraphs, all_images, 'PC_Hardware_News_Compilation.pdf')


Streaming output truncated to the last 5000 lines.
Error extracting text from image /u/styles/large_responsive_no_watermark_/public/inline-images/gn-patreon-1000x250%20copy_0.png.webp?itok=4RZbJENx: Invalid URL '/u/styles/large_responsive_no_watermark_/public/inline-images/gn-patreon-1000x250%20copy_0.png.webp?itok=4RZbJENx': No scheme supplied. Perhaps you meant https:///u/styles/large_responsive_no_watermark_/public/inline-images/gn-patreon-1000x250%20copy_0.png.webp?itok=4RZbJENx?
Error extracting text from image /u/styles/card_teaser/public/2024-08/vlcsnap-2024-08-05-16h16m19s213.jpg.webp: Invalid URL '/u/styles/card_teaser/public/2024-08/vlcsnap-2024-08-05-16h16m19s213.jpg.webp': No scheme supplied. Perhaps you meant https:///u/styles/card_teaser/public/2024-08/vlcsnap-2024-08-05-16h16m19s213.jpg.webp?
Error extracting text from image /u/styles/card_teaser/public/2024-07/vlcsnap-2024-07-29-16h00m47s545.jpg.webp: Invalid URL '/u/styles/card_teaser/public/2024-07/vlcsnap-2024-07-29-